## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,55.56,86,90,2.30,moderate rain
1,1,Sokoni,TZ,-6.4333,39.5500,75.06,75,8,9.33,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,71.76,88,75,5.75,broken clouds
3,3,Kathu,TH,7.9178,98.3332,92.62,76,72,7.02,broken clouds
4,4,Chuy,UY,-33.6971,-53.4616,51.57,81,18,7.52,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sokoni,TZ,-6.4333,39.5500,75.06,75,8,9.33,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,71.76,88,75,5.75,broken clouds
5,5,Meulaboh,ID,4.1363,96.1285,81.52,83,95,4.79,overcast clouds
6,6,Imbituba,BR,-28.2400,-48.6703,64.29,92,10,9.48,clear sky
7,7,Mahebourg,MU,-20.4081,57.7000,70.09,94,75,9.22,light rain
8,8,Albany,US,42.6001,-73.9662,67.95,94,100,2.26,overcast clouds
11,11,Mattru,SL,7.6244,-11.8332,75.65,97,99,1.95,overcast clouds
12,12,Olinda,BR,-8.0089,-34.8553,73.13,78,0,5.32,clear sky
13,13,Adrar,MR,20.5022,-10.0711,85.69,28,41,18.72,scattered clouds
14,14,Cherskiy,RU,68.7500,161.3000,69.55,23,99,8.63,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sokoni,TZ,75.06,clear sky,-6.4333,39.5500,
2,Vaini,TO,71.76,broken clouds,-21.2000,-175.2000,
5,Meulaboh,ID,81.52,overcast clouds,4.1363,96.1285,
6,Imbituba,BR,64.29,clear sky,-28.2400,-48.6703,
7,Mahebourg,MU,70.09,light rain,-20.4081,57.7000,
8,Albany,US,67.95,overcast clouds,42.6001,-73.9662,
11,Mattru,SL,75.65,overcast clouds,7.6244,-11.8332,
12,Olinda,BR,73.13,clear sky,-8.0089,-34.8553,
13,Adrar,MR,85.69,scattered clouds,20.5022,-10.0711,
14,Cherskiy,RU,69.55,overcast clouds,68.7500,161.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df.count()

City                   456
Country                456
Max Temp               456
Current Description    456
Lat                    456
Lng                    456
Hotel Name             410
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   410
Country                410
Max Temp               410
Current Description    410
Lat                    410
Lng                    410
Hotel Name             410
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map.
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))